In [ ]:
from PIL import Image
import numpy as np


In [ ]:
def getblocks(image: np.ndarray, blockshape: tuple, moveAxis: bool = True, info: bool = False, addChannel: bool = True) -> np.ndarray:
    '''
    takes the array of image in grey= 2D and in RGB = 3D
    takes the numpy array and converts it the the blocks in the fastest way
    '''
    if(info):
        print("Image Shape:", image.shape)
        print("Block Shape:", blockshape)

    oldshape = list(image.shape)
    if addChannel and len(image.shape) == 2:
        mode = "grey"
        image = image.reshape((*image.shape, 1))
    else:
        mode = "color"

    if addChannel:
        img_height, img_width, channels = image.shape
    else:
        img_height, img_width = image.shape

    tile_height, tile_width = blockshape

    if addChannel:
        shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width, channels
    else:
        shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width

    def printinfo():
        print("Old Shape:", oldshape)
        print("Image Shape:", image.shape)
        print("Block Shape:", blockshape)
        print("New Shape Initial:", shp)
        print("img_height % tile_height != 0 :", img_height % tile_height != 0)
        print("img_width % tile_width != 0 :", img_width % tile_width != 0)

    if img_height % tile_height != 0 or img_width % tile_width != 0:
        print("warning: Block size is not fit for the image!")
        printinfo()

    if(info):
        printinfo()

    tiled_array = image.reshape(shp)
    tiled_array = tiled_array.swapaxes(1, 2)

    if moveAxis:
        if(addChannel):
            tiled_array = tiled_array.reshape(-1,
                                              *(tile_height, tile_width, channels))
            tiled_array = np.moveaxis(tiled_array, source=len(
                tiled_array.shape)-1, destination=1)
        else:
            tiled_array = tiled_array.reshape(-1, *(tile_height, tile_width))

    return tiled_array


In [ ]:
def combineBlocks(tiled_array: np.ndarray, imageshape: tuple, blockshape: tuple, movedAxis: bool = True, channel: bool = True) -> np.ndarray:

    if channel:
        if len(imageshape) == 2:
            mode = "grey"
            imageshape = *imageshape, 1
        else:
            mode = "color"

    if channel:
        img_height, img_width, channels = imageshape
    else:
        img_height, img_width = imageshape

    tile_height, tile_width = blockshape

    if movedAxis:
        image = tiled_array.copy()
        if(channel):
            image = image.reshape(img_height//tile_height, tile_height,
                                  img_width//tile_width, tile_width, channels)
            swapaxisShape = list(image.shape)
            swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
            image = image.reshape(swapaxisShape)
            image = image.swapaxes(1, 2)
        else:
            f = image.reshape(img_height//tile_height, tile_height,
                              img_width//tile_width, tile_width)
            swapaxisShape = list(f.shape)
            swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
            tmp = f.reshape(swapaxisShape)
            image = tmp.swapaxes(1, 2)
    else:
        image = tiled_array
        # I haven't completed this else case. Btw we aren't using this case lol :)

    return image.reshape(imageshape)


In [ ]:
def set_bit(value:int, index:int, x:int):
    # """Set the index:th bit of v to 1 if x is truthy, else to 0, and return the new value."""
    # mask = 1 << index   # Compute mask, an integer with just bit 'index' set.
    # # Clear the bit indicated by the mask (if x is False)
    # value &= ~mask
    # if x:
    #     # If x was True, set the bit indicated by the mask.
    #     value |= mask
    # return value            # Return the result, we're done.
    def set_bit2(value, bit):
        return value | (1 << bit)

    def clear_bit(value, bit):
        return value & ~(1 << bit)

    if x:
        return set_bit2(value, index)
    else:
        return clear_bit(value, index)


def get_bit(value, index):
    localVal=int(value)
    if localVal & (1 << index):
        return 1
    else:
        return 0


In [ ]:
import hashlib


In [ ]:
HASH_SIZE = 16


In [ ]:
def set_lsb_zero(num: np.ndarray):
    '''
    Clearing the first two LSB of ndarray
    '''
    return set_bit(set_bit(num, 0, 0), 1, 0)


'''
Data is one 16x16 block converted into four 8x8 blocks
'''


def hash_block(data: np.ndarray, key: str = None, digest_size=HASH_SIZE, extras=[]):

    local = data.copy().astype(np.int8)  # copying to avoid overighting lsb
    local = set_lsb_zero(local)  # setting last 8x8 blocks lsb zero
    if key is None:
        h = hashlib.blake2b(digest_size=digest_size)
    else:
        h = hashlib.blake2b(key=key.encode())
    h.update(local.data)
    for extra in extras:
        h.update(extra.encode())
    return h


In [ ]:
def hexToDec(hexStr):
    return np.fromiter((int(x, 16) for x in hexStr), dtype=np.uint8)


def binToNp(binStr):
    return np.frombuffer(binStr, dtype=np.uint8)


In [ ]:
image = np.arange(1, (5*5)+1).reshape(5, 5)
image


In [ ]:
# get_bit(image[:,0],0)
np.fromiter((get_bit(num, 0) for num in 
             image.flat), dtype=image.dtype)

In [ ]:
set_bit(image[:,0],0,1)

In [ ]:
hashes = np.zeros((*image.shape, HASH_SIZE*8), dtype=object)
hashes.shape, hashes


In [ ]:
onHashNumbers = []
onExtractNums = []


In [ ]:
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        hashes[i][j] = np.unpackbits(binToNp(hash_block(
            image[i, j]).digest()))
        onHashNumbers.append(set_bit(set_bit(image[i][j].copy(), 0, 0), 1, 0))


hashes.shape, hashes[0][0], hashes


In [ ]:
# reshaping it into 2bits in last for better placement
hashbits = hashes.reshape(
    hashes.shape[0], hashes.shape[1] * hashes.shape[2]//2, 2)
hashbits.shape


In [ ]:
set_bit(2,0,0)

In [ ]:
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        number = image[i,:].flat[j]
        image[i, :].flat[j] = set_bit(
            set_bit(number, 0, hashbits[i][j][0]), 1, hashbits[i][j][1])


In [ ]:
extractedHashes = np.zerWos(hashbits.shape, dtype=object)
extractedHashes.shape, extractedHashes


In [ ]:
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        number = image[i][j]
        extractedHashes[i][j] = get_bit(number, 0)
        extractedHashes[i][j] = get_bit(number, 1)
        onExtractNums.append(set_bit(set_bit(image[i][j].copy(), 0, 0), 1, 0))


In [ ]:
extractedHashes.shape, hashbits.shape


In [ ]:
extractedHashes == hashbits


In [ ]:
(extractedHashes == hashbits).all()

In [ ]:
onExtractNums


In [ ]:
onHashNumbers
